# **FriskvårdsCentrum Nordics**

``Bransch:`` Gymkedja

``Beskrivning:``Driver gym och gruppträning på flera orter. Datasetet innehåller bokningar av träningspass, medlemsinfo och feedback efter passen.

**Viktigt:** Kör `etl_pipeline.py` innan du läser/validerar i notebook så att huvuddata sparas med `dataset="main"`.

In [1]:
# Importera bibliotek
import pandas as pd
import numpy as np
import sqlite3

## Inläsning av rådata och EDA

In [2]:
# Läs in dataset
df = pd.read_csv('friskvard_data.csv')

# Skriv ut datasetets rader, kolumner och information
print("Datasetets storlek (rader, kolumner):", df.shape)
print("\nInformation:")
df.info()

Datasetets storlek (rader, kolumner): (3075, 18)

Information:
<class 'pandas.DataFrame'>
RangeIndex: 3075 entries, 0 to 3074
Data columns (total 18 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   bokning_id         3075 non-null   str    
 1   medlem_id          3075 non-null   str    
 2   medlemstyp         3075 non-null   str    
 3   medlem_startdatum  3075 non-null   str    
 4   medlem_slutdatum   3075 non-null   str    
 5   månadskostnad      3075 non-null   int64  
 6   födelseår          2977 non-null   float64
 7   pass_id            3075 non-null   str    
 8   passnamn           3075 non-null   str    
 9   anläggning         2978 non-null   str    
 10  instruktör         2955 non-null   str    
 11  bokningsdatum      3075 non-null   str    
 12  passdatum          3075 non-null   str    
 13  passtid            3075 non-null   str    
 14  status             3075 non-null   str    
 15  feedback_text      1

In [3]:
# Visa fem första raderna i dataset
df.head()

,bokning_id,medlem_id,medlemstyp,medlem_startdatum,medlem_slutdatum,månadskostnad,födelseår,pass_id,passnamn,anläggning,instruktör,bokningsdatum,passdatum,passtid,status,feedback_text,feedbackdatum,feedback_betyg
0,BOK-000001,MED-10158,Premium,2023-07-29,2025-07-03,599,2006.0,PASS-2024-10-01-001,Yoga,GÖTEBORG CENTRUM,Maria Santos,2024-09-24,2024-10-01,11:00,Genomförd,Bästa yogapasset jag varit på. Kommer tillbaka!,2024-10-02,4.0
1,BOK-000002,MED-10229,Premium,2023-08-01,2025-06-26,599,1984.0,PASS-2024-12-04-002,Cykel,MALMÖ CENTRUM,Johan Bergström,2024-11-27,2024-12-04,10:00,Genomförd,Professionellt och välorganiserat pass.,2024-12-05,5.0
2,BOK-000003,MED-10223,Student,2023-12-27,2025-05-12,249,2006.0,PASS-2024-11-09-003,Styrketräning,NaN,Erik Johansson,2024-11-08,2024-11-09,18:00,Genomförd,NaN,NaN,NaN
3,BOK-000004,MED-10110,Bas,"July 03, 2023",2025-03-24,349,2006.0,PASS-2024-09-04-004,Pilates,Göteborg Centrum,Maria Santos,2024-09-01,4 september 2024,07:00,Klar,Professionellt och välorganiserat pass.,2024-09-02,5.0
4,BOK-000005,MED-10022,Premium,2022-11-05,2025-06-01,599,1992.0,PASS-2024-10-29-005,Pilates,Västerås,Klara Svensson,2024-10-26,2024-10-29,20:30,Genomförd,Bästa pilatespasset jag varit på. Kommer tillb...,2024-10-30,5.0


In [4]:
# Visa fem sista raderna i dataset
df.tail()

,bokning_id,medlem_id,medlemstyp,medlem_startdatum,medlem_slutdatum,månadskostnad,födelseår,pass_id,passnamn,anläggning,instruktör,bokningsdatum,passdatum,passtid,status,feedback_text,feedbackdatum,feedback_betyg
3070,BOK-002996,MED-10037,Premium,2024-03-13,2024-10-12,599,2003.0,PASS-2024-10-24-2996,Boxning,Stockholm Kungsholmen,Marcus Ek,2024-10-19,2024-10-24,17:30,Genomförd,NaN,NaN,NaN
3071,BOK-002997,MED-10199,Bas,2022-07-25,2024-12-09,349,2004.0,PASS-2024-12-03-2997,Styrketräning,Örebro,NaN,2024-11-30,2024/12/03,18:30,No-show,NaN,NaN,NaN
3072,BOK-002998,MED-10073,Premium,2023-02-19,2024-08-26,-349,1982.0,PASS-2024-10-03-2998,Boxning,Malmö Centrum,Johan Bergström,2024-09-26,2024-10-03,19:00,Genomförd,NaN,NaN,NaN
3073,BOK-002999,MED-10151,Bas,2023-11-10,2024-09-09,349,1996.0,PASS-2024-07-03-2999,Boxning,Malmö Centrum,Johan Bergström,2024-06-29,2024-07-03,07:30,Genomförd,Instruktören var superbra på att korrigera tek...,2024-07-03,4.0
3074,BOK-003000,MED-10320,bas,2024-05-06,2025-07-07,349,NaN,PASS-2024-12-18-3000,Pilates,Västerås,Klara Svensson,2024-12-17,2024-12-18,18:30,Klar,NaN,NaN,NaN


In [5]:
# Kolla om bokning_id är unikt för varje rad
print("Antal rader:", len(df))
print("Unika bokning_id:", df['bokning_id'].nunique())

Antal rader: 3075
Unika bokning_id: 3000


In [6]:
# Visa antal dubletter
duplicate_count = df.duplicated().sum()
print("Antal dubletter i dataset:", duplicate_count)

Antal dubletter i dataset: 75


Boknings_id - Är troligtvis grainet, om dubletterna tas bort blir alla rader i bokning_id unika och matchar antalet raderna i dataset.

In [7]:
# Visa fördelningen av värden i månadskostnad innan rengöring
df['månadskostnad'].value_counts()

månadskostnad
 349    1468
 599     966
 249     588
 0        20
-449      18
-349      15
Name: count, dtype: int64

Vad är minus och nollbeloppen?

In [8]:
# Antal unika medlemmar med 0 kr i månadskostnad
medlemmar_zero_cost = df[df['månadskostnad'] == 0]['medlem_id'].nunique()
print("\nAntal medlemmar med 0 kr i månadskostnad:", medlemmar_zero_cost)

# Max antal 0‑poster för en medlem
max_zero_cost = df[df['månadskostnad'] == 0].groupby('medlem_id').size().max()
print("Max antal nollbelopp per medlem:", max_zero_cost)


Antal medlemmar med 0 kr i månadskostnad: 20
Max antal nollbelopp per medlem: 1


In [9]:
# Sammanställning av minusbelopp per status och per medlem
neg = df[df['månadskostnad'] < 0]

print("Minusbelopp per status:")
print(neg.groupby('status')['månadskostnad'].agg(['count','sum']).sort_values('count', ascending=False))

# Max antal minusposter för en medlem
max_minus = neg.groupby('medlem_id').size().max()
print("\nMax antal minusposter för en medlem:", max_minus)

Minusbelopp per status:
           count   sum
status                
Genomförd     23 -9327
No-show        4 -1596
GENOMFÖRD      2  -798
Avbokad        2  -798
avbokad        1  -449
genomförd      1  -349

Max antal minusposter för en medlem: 1


In [10]:
# Visa medlemstyp och status för varje medlem med minusbelopp
neg_members = neg[['medlem_id', 'medlemstyp', 'status']].drop_duplicates()
print("\nMedlemmar med minusbelopp:")
print(neg_members)


Medlemmar med minusbelopp:
      medlem_id  medlemstyp     status
100   MED-10305     STUDENT  Genomförd
113   MED-10243     Premium  GENOMFÖRD
156   MED-10052     Student  Genomförd
302   MED-10085         Bas  Genomförd
486   MED-10281         Bas  genomförd
610   MED-10369     STUDENT  Genomförd
695   MED-10172     Premium  Genomförd
710   MED-10147         Bas    Avbokad
725   MED-10214         Bas    Avbokad
1007  MED-10128     Student  Genomförd
1050  MED-10078     Premium  Genomförd
1174  MED-10333         bas  Genomförd
1254  MED-10040     Premium  Genomförd
1271  MED-10348     Premium  Genomförd
1315  MED-10260         Bas  Genomförd
1743  MED-10186         bas    No-show
1756  MED-10254     Premium    avbokad
1832  MED-10101         Bas  Genomförd
1894  MED-10277         Bas  Genomförd
1918  MED-10353     Premium  Genomförd
1929  MED-10197         Bas  Genomförd
2185  MED-10270  Studerande  Genomförd
2255  MED-10004     Premium  Genomförd
2279  MED-10250         Bas    No-sh

**Flagga minusbelopp de verkar inte vara felinslag**
- Minusbelopp = Tyder på återbetalning/justering
- 0 kr belopp = Tyder på gratis månad (kanske personal/familjmedlemskap)

In [11]:
# Visa info för 'födelseår'
df['födelseår'].info()

<class 'pandas.Series'>
RangeIndex: 3075 entries, 0 to 3074
Series name: födelseår
Non-Null Count  Dtype  
--------------  -----  
2977 non-null   float64
dtypes: float64(1)
memory usage: 24.2 KB


In [12]:
# Visa fördelningen av värden i 'medlemstyp'
df['medlemstyp'].value_counts()

medlemstyp
Bas           1321
Premium        884
Student        510
Grund           46
bas             45
Basic           44
BAS             40
STUDENT         35
PREMIUM         29
Studerande      28
premium         26
Gold            24
Plus            22
student         21
Name: count, dtype: int64

In [13]:
# Visa fördelningen av värden i 'anläggning'
df['anläggning'].value_counts()

anläggning
Malmö Västra Hamnen      238
Västerås                 227
Uppsala                  222
Göteborg Hisingen        221
Stockholm City           221
Lund                     206
Stockholm Södermalm      204
Malmö Centrum            203
Göteborg Centrum         201
Stockholm Kungsholmen    194
Linköping                193
Örebro                   188
LUND                      23
Linköping C               21
STOCKHOLM SÖDERMALM       20
UPPSALA                   20
VÄSTERÅS                  19
Malmö VH                  18
LINKÖPING                 17
Sthlm Södermalm           17
Uppsala C                 17
Södermalm                 16
Göteborg C                16
GÖTEBORG CENTRUM          15
Sthlm Kungsholmen         15
Örebro C                  15
Malmö C                   15
Gbg Centrum               15
Lund C                    14
Malmö city                14
STOCKHOLM CITY            13
Sthlm City                13
ÖREBRO                    13
Gbg Hisingen              13
Väs

In [14]:
# Visa fördelningen av värden i 'status'
df['status'].value_counts()

status
Genomförd        2133
Avbokad           302
No-show           272
genomförd          79
Deltog             75
GENOMFÖRD          72
Klar               56
no-show            16
NO-SHOW            15
Struken            13
Ej närvarande      13
AVBOKAD             9
avbokad             8
Cancelled           7
Missad              5
Name: count, dtype: int64

In [15]:
# Visa fördelningen av värden i 'passnamn'
df['passnamn'].value_counts()

passnamn
Spinning          545
HIIT              527
Yoga              470
Pilates           318
Styrketräning     303
Boxning           216
Dans              204
Intervall          30
H.I.I.T            29
pilates            27
Core               25
Vinyasa            24
PILATES            22
yoga               22
SPINNING           22
Cykel              21
Zumba              20
Högintensiv        20
spinning           19
Hatha Yoga         19
Spin               18
YOGA               18
Indoor Cycling     18
Boxing             16
hiit               14
styrka             13
DANS               13
Styrkepass         11
Dance              10
Fightpass          10
Gympass            10
STYRKA             10
dans                8
BOXNING             8
boxning             8
Strength            7
Name: count, dtype: int64

In [16]:
# Visa kolumner med datum
df[['medlem_startdatum', 'medlem_slutdatum', 'bokningsdatum', 'passdatum', 'feedbackdatum']].head(10)

,medlem_startdatum,medlem_slutdatum,bokningsdatum,passdatum,feedbackdatum
0,2023-07-29,2025-07-03,2024-09-24,2024-10-01,2024-10-02
1,2023-08-01,2025-06-26,2024-11-27,2024-12-04,2024-12-05
2,2023-12-27,2025-05-12,2024-11-08,2024-11-09,NaN
3,"July 03, 2023",2025-03-24,2024-09-01,4 september 2024,2024-09-02
4,2022-11-05,2025-06-01,2024-10-26,2024-10-29,2024-10-30
5,2022-08-10,2024-09-19,2024-09-11,2024/09/18,NaN
6,2022/11/28,2025-05-04,2024-08-13,2024-08-16,NaN
7,2024-03-19,2025-04-20,2024-07-04,2024-07-09,NaN
8,2022-11-04,2025-02-14,2024-12-26,2024-12-28,NaN
9,2022-07-20,2024-09-03,2024-08-02,2024-08-05,2024-08-06


In [17]:
# Visa fördelning av värden i 'passtid'
df['passtid'].value_counts()

passtid
17:00    364
18:00    358
17:30    327
18:30    266
12:00    222
07:00    212
19:00    207
07:30    174
16:00    157
20:00    154
10:00    151
06:30    114
08:00    110
11:00     98
09:00     95
20:30     66
Name: count, dtype: int64

## Datatvätt:

- Ta bort dubletter
- Flagga negativa belopp och skapa absolutbelopp: ``'månadskostnad'``
- Konvertera till int: ``födelseår``
- Standardisera namngivning och bestäm textstorlek: ``'medlemstyp'``, ``'anläggning'``,``'status'``,``'passnamn'``
- Hantera Null-värden: ``'anläggning', 'instruktör' och 'feedback_text'``
- Konvertera till datetime: ``'medlem_startdatum', 'medlem_slutdatum', 'bokningsdatum', 'passdatum' och 'feedbackdatum'``
- Konvertera till time: ``'passtid'``
- Konvertera till kategori: ``'medlemstyp', 'anläggning', 'status', 'passnamn' och 'instruktör'``

In [21]:
# Importera funktioner från etl_pipeline.py
from etl_pipeline import (
    anlaggning_map,
    medlemstyp_map,
    passnamn_map,
    status_map,
    sv_months,
    clean_månadskostnad,
    clean_födelseår,
    clean_medlemstyp,
    clean_anläggning,
    clean_status,
    clean_passnamn,
    clean_null_values,
    clean_date,
    clean_passtid,
    convert_to_category,
    transform_data,
    load_dataset_to_db,
 )

## Dokumentation

**Sammanfattning av kolumner och transformationer efter datarengöring(data.csv):**

| Kolumn | Datatyp | Beskrivning | Transformationer |
|--------|---------|-------------|------------------|
| **bokning_id, medlem_id, pass_id** | str | Unikt ID för varje bokning, medlem, pass | Ingen ändring |
| **födelseår** | Int | Medlemmens födelseår | Konverterad till Int (hanterar NaN som `<NA>`) |
| **medlemstyp** | category | Typ av medlemskap | Standardiserad till Title Case. Gruppering: 'Grund'/'Basic' → 'Bas', 'Studerande' → 'Student', 'Gold'/'Plus' → 'Premium' |
| **månadskostnad** | int | Månadskostnad för medlemskap | Negativa belopp flaggade (neutral) och absolutbelopp skapat för analys |
| **medlem_startdatum** | datetime | När medlemskap startade | Konverterad från US-format (MM/DD/YYYY) till datetime |
| **medlem_slutdatum** | datetime | När medlemskap slutar | Konverterad från US-format (MM/DD/YYYY) till datetime |
| **passdatum** | datetime | Datum för passet | Konverterad från US-format (MM/DD/YYYY) till datetime |
| **bokningsdatum** | datetime | När bokningen gjordes | Konverterad från US-format (MM/DD/YYYY) till datetime |
| **passtid** | object (time) | Tid för passet (HH:MM) | Konverterad till time-format |
| **anläggning** | category | Gymlokal där passet bokades | Standardiserad till Title Case. Förkortningar ersatta (t.ex. 'Sthlm City' → 'Stockholm City', 'Gbg' → 'Göteborg'). NaN fylld med 'Okänd' |
| **passnamn** | category | Namn på träningspasset | Standardiserad till Title Case. Gruppering: HIIT-varianter → 'Hiit', Styrka-varianter → 'Styrketräning', Cykel → 'Spinning', Dans-varianter → 'Dans', Yoga-varianter → 'Yoga', Boxning-varianter → 'Boxning' |
| **instruktör** | category | Namn på instruktören | NaN fylld med 'Okänd' |
| **status** | category | Bokningsstatus | Standardiserad till Title Case. Gruppering: 'Deltog'/'Klar' → 'Genomförd', 'Struken'/'Cancelled' → 'Avbokad', 'Ej Närvarande'/'Missad'/'No Show' → 'No-Show' |
| **feedback_betyg** | float | Betyg 1-5 från medlemmen | Ingen ändring (NaN behålls för saknade betyg) |
| **feedback_text** | object | Fritextkommentar från medlemmen | NaN fylld med tom sträng ('') för bättre textanalys |
| **feedbackdatum** | datetime | När feedbacken lämnades | Konverterad från US-format (MM/DD/YYYY) till datetime |

**Övergripande transformationer:**
- Dubbletter: Kontrollerad och borttagen
- Kategoriska kolumner: Konverterade till `category` datatyp för minneseffektivitet
- Saknade värden: Hanterade strategiskt beroende på kolumntyp
- Datum i US-format (MM/DD/YYYY): Konverterade till datetime

**Tolkning av avvikande belopp (EDA):**
- **0 kr i månadskostnad**: Tyder sannolikt på gratisperiod.
- **Negativa belopp**: Oklassade negativa transaktioner (t.ex. korrigeringar), behålls för analys.

**Grain: Bokning (bokning_id)**
- **Primärnyckel:** bokning_id (unikt för varje rad)
- **En rad representerar:** En enskild bokning av ett träningspass av en medlem
- **Relation:** En medlem (medlem_id) kan ha flera bokningar

### Valideringsdata:

**Sammanfattning av kolumner och transformationer (valideringsdata):**

| Kolumn | Datatyp (rengjord) | Beskrivning | Transformationer |
|--------|-------------------|-------------|------------------|
| **bokning_id, medlem_id, pass_id** | str | Unikt ID för bokning, medlem, pass | Ingen ändring |
| **födelseår** | Int | Medlemmens födelseår | Konverterad till Int (NaN → `<NA>`) |
| **medlemstyp** | category | Typ av medlemskap | Standardiserad (Title Case), mappning till Bas/Student/Premium |
| **månadskostnad** | int | Månadskostnad | Negativa belopp flaggas, absolutbelopp skapas |
| **medlem_startdatum** | datetime | Startdatum | Konverterad från blandade format till datetime |
| **medlem_slutdatum** | datetime | Slutdatum | Konverterad från blandade format till datetime |
| **passdatum** | datetime | Passdatum | Konverterad från blandade format till datetime |
| **bokningsdatum** | datetime | Bokningsdatum | Konverterad från blandade format till datetime |
| **passtid** | object (time) | Tid för passet | Konverterad till time |
| **anläggning** | category | Anläggning | Standardiserad, förkortningar ersatta, NaN → 'Okänd' |
| **passnamn** | category | Passnamn | Standardiserad och grupperad (Hiit, Styrketräning, Spinning, Dans, Yoga, Boxning) |
| **instruktör** | category | Instruktör | NaN → 'Okänd' |
| **status** | category | Status | Standardiserad och grupperad (Genomförd, Avbokad, No-Show) |
| **feedback_betyg** | float | Betyg 1–5 | Oförändrad, NaN behålls |
| **feedback_text** | object | Fritext | NaN → '' |
| **feedbackdatum** | datetime | Feedbackdatum | Konverterad från blandade format till datetime |

**Övergripande transformationer:**
- Dubbletter borttagna
- Kategoriska kolumner konverterade till `category`
- Saknade värden hanterade per kolumn
- Datum normaliserade till datetime

In [ ]:
# Visa ändringar efter datatransformering i valideringsdataset
from etl_pipeline import transform_data

df_raw_validtation = pd.read_csv('friskvard_validation.csv')
# Visa information före
print(df_raw_validtation.info())

df_raw_cleaned_validation = transform_data(df_raw_validtation)
# Visa information efter
print(df_raw_cleaned_validation.info())

<class 'pandas.DataFrame'>
RangeIndex: 461 entries, 0 to 460
Data columns (total 18 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   bokning_id         461 non-null    str    
 1   medlem_id          461 non-null    str    
 2   medlemstyp         461 non-null    str    
 3   medlem_startdatum  461 non-null    str    
 4   medlem_slutdatum   461 non-null    str    
 5   månadskostnad      461 non-null    int64  
 6   födelseår          446 non-null    float64
 7   pass_id            461 non-null    str    
 8   passnamn           461 non-null    str    
 9   anläggning         449 non-null    str    
 10  instruktör         439 non-null    str    
 11  bokningsdatum      461 non-null    str    
 12  passdatum          461 non-null    str    
 13  passtid            461 non-null    str    
 14  status             461 non-null    str    
 15  feedback_text      181 non-null    str    
 16  feedbackdatum      181 non-null    st